# Project 4: Port Scanner Using Python

# Problem Statement:
In network security, open or vulnerable ports on servers and devices represent significant entry points for malicious actors. 
Understanding which ports are open and what services they are running is a fundamental step in assessing a 
system's security posture and identifying potential weaknesses.

# Objective:
To develop a command-line based port scanner in Python capable of identifying open ports on a given IP address or hostname. 
This tool will serve as a basic utility for network reconnaissance and initial vulnerability assessment.

# Technologies Utilized:
* **Python:** The core programming language.
* **`socket` module:** Python's low-level networking interface, used to establish connections to target ports.
* **`threading` module:** Employed to implement concurrency, allowing the scanner to check multiple ports simultaneously, 
     thereby significantly improving performance.
* **`sys` and `optparse` (or `argparse`) modules:** For handling command-line arguments to make the script user-friendly and configurable.

# Expected Outcome:
A robust and efficient Python script that, when executed from the command line, takes a target IP/hostname and a range of ports as input, 
and then outputs a clear list of all detected open ports. 
This will aid in understanding accessible network services.

# Library Imports and Initial Setup

This section imports the necessary Python modules for our port scanner.
* `socket`: Crucial for establishing network connections to check port status.
* `threading`: Allows us to run multiple port checks concurrently, speeding up the scanning process.
* `sys`: Used for interacting with the system, particularly for exiting the script in case of errors.
* `optparse`: This module helps in parsing command-line arguments (like the target IP and port range) provided by the user.

In [1]:
import socket
import threading
import sys
from optparse import OptionParser

# A lock object to manage access to shared resources (like printing to console)
# when multiple threads are running. This prevents messy output.
screenLock = threading.Semaphore(value=1)

# List to store open ports found
open_ports = []

print("Required libraries imported and initial setup complete.")

Required libraries imported and initial setup complete.


# Port Scanning Function (`connScan`)

This is the core function that attempts to connect to a specific port on a given target.
* It tries to create a `socket` connection to the `tgtHost` (target host) on `tgtPort` (target port).
* If the connection is successful, it means the port is open. It then prints this information and records the port.
* If the connection fails (e.g., connection refused, timeout), it means the port is likely closed or filtered.
* It includes a timeout to prevent the scanner from hanging indefinitely on non-responsive ports.

In [ ]:
def connScan(tgtHost, tgtPort):
    """
    Attempts to connect to a specific port on the target host.
    If successful, the port is considered open.
    """
    try:
        # Create a socket object
        # AF_INET for IPv4, SOCK_STREAM for TCP
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

        # Set a timeout for the connection attempt (e.g., 1 second)
        s.settimeout(1)

        # Attempt to connect to the target host and port
        result = s.connect_ex((tgtHost, tgtPort)) # connect_ex returns an error indicator

        # If result is 0, connection was successful, meaning the port is open
        if result == 0:
            screenLock.acquire() # Acquire lock before printing to prevent mixed output from threads
            print(f"[+] {tgtPort}/tcp open")
            open_ports.append(tgtPort) # Add open port to our list
        
        screenLock.release() # Release lock after printing

    except Exception as e:
        # If any other error occurs during connection, print it
        screenLock.acquire()
        print(f"[-] Could not connect to {tgtPort}/tcp: {e}")
        screenLock.release()
    finally:
        # Ensure the socket is closed whether connection succeeded or failed
        s.close()

# Target Host Scanning Function (`portScan`)

This function coordinates the scanning process for a single target host across a list of ports.
* First, it resolves the `tgtHost` (which could be a domain name like "google.com") into an actual IP address. This is important because `socket.connect_ex` needs an IP.
* It then creates a new thread for each port in the `tgtPorts` list.
* Each thread calls the `connScan` function, allowing multiple ports to be checked concurrently. This makes the scanner much faster than checking ports one by one.

In [ ]:
def portScan(tgtHost, tgtPorts):
    """
    Scans a list of ports on the target host.
    """
    try:
        # Resolve hostname to IP address
        tgtIP = socket.gethostbyname(tgtHost)
    except socket.gaierror:
        print(f"[-] Cannot resolve '{tgtHost}': Unknown Host")
        return # Exit if host cannot be resolved

    try:
        # Print info about the target IP
        print(f"\n[+] Scan Results for: {tgtIP}")
    except:
        # Handles cases where gethostbyname might succeed but hostname is weird
        print(f"\n[+] Scan Results for: {tgtHost}")

    # For each target port, create and start a new thread
    for tgtPort in tgtPorts:
        t = threading.Thread(target=connScan, args=(tgtHost, int(tgtPort)))
        t.start()

# Main Execution Logic and Command-line Interface

This section contains the main part of the script that gets executed when the notebook cell is run. It sets up the command-line interface using `OptionParser` to accept input from the user.

**How to use this:**
* When you run this cell in Jupyter, it simulates a command-line execution.
* You will be prompted to enter the target host and ports within the notebook output.
* Alternatively, if this code were saved as a `.py` file, you would run it from your terminal like this:
    `python your_port_scanner.py -H <target_ip_or_hostname> -p <port1,port2,port3...>`
    or
    `python your_port_scanner.py -H <target_ip_or_hostname> -p <start_port>-<end_port>`

**Example inputs you can try in the prompt (for testing purposes, use safe targets):**
* Target Host: `scanme.nmap.org` (This is a legitimate target provided by Nmap for testing scanners)
* Target Ports: `21,22,80,443,3389` (Common ports for FTP, SSH, HTTP, HTTPS, RDP)

**Important Note for Testing:**
* **Do NOT scan random public websites or IPs without explicit permission.** Scanning without permission can be illegal and unethical.
* `scanme.nmap.org` is a safe and permitted target for testing.
* You can also scan `localhost` (your own machine) to see ports open on your system (e.g., `127.0.0.1` and ports like `8000` if you have a local web server running, or `445` for SMB).

In [6]:
def main():
    # Setup OptionParser to handle command-line arguments
    parser = OptionParser("usage %prog -H <target host> -p <target port(s)>")
    parser.add_option('-H', dest='tgtHost', type='string', help='specify target host')
    parser.add_option('-p', dest='tgtPorts', type='string', help='specify target port(s) separated by comma or a range (e.g., "21,22,80" or "1-100")')

    # Simulate parsing arguments or get them interactively in Jupyter
    # In a real script, this would be: (options, args) = parser.parse_args()
    # For Jupyter, we'll prompt the user if no arguments are passed.

    target_host_input = input("Enter Target Host (e.g., scanme.nmap.org or 127.0.0.1): ")
    target_ports_input = input("Enter Target Port(s) (e.g., 21,80,443 or 1-100): ")

    if not target_host_input or not target_ports_input:
        print(parser.usage)
        sys.exit(0) # Exit if essential info is missing

    tgtHost = target_host_input
    
    # Parse the ports input
    if '-' in target_ports_input:
        try:
            start_port, end_port = map(int, target_ports_input.split('-'))
            tgtPorts = [str(p) for p in range(start_port, end_port + 1)]
        except ValueError:
            print("[-] Invalid port range. Use 'start-end' format.")
            sys.exit(0)
    else:
        tgtPorts = str(target_ports_input).split(',')


    if (tgtHost == None) or (tgtPorts[0] == None):
        print(parser.usage)
        exit(0)
    
    portScan(tgtHost, tgtPorts)

# This ensures main() is called when the script is executed
if __name__ == '__main__':
    print("Port Scanner initiated. Please provide input in the prompts below.")
    main()

Port Scanner initiated. Please provide input in the prompts below.


Enter Target Host (e.g., scanme.nmap.org or 127.0.0.1):  scanme.nmap.org
Enter Target Port(s) (e.g., 21,80,443 or 1-100):  21,22,80,443



[+] Scan Results for: 45.33.32.156
[+] 21/tcp open
[+] 22/tcp open
[+] 80/tcp open


# Conceptual Output / Scan Results

After running the port scanner and providing a target host and ports, the script will output the status of each port. The output will dynamically appear below the code cell as the scan progresses.

**Example Scan Output (for `scanme.nmap.org` scanning ports `21, 22, 80, 443`):**

This output clearly indicates which specified ports are open (e.g., 21, 22, 80) on the target host `45.33.32.156` (which is `scanme.nmap.org`). This information is vital for understanding accessible services and potential entry points on a network.


# Conclusion - Project 4: Port Scanner Using Python

This project successfully developed a basic yet effective port scanner using Python's `socket` and `threading` modules. The tool is capable of identifying open TCP ports on a specified target IP address or hostname, enhancing the efficiency of the scan through multithreading. By providing a clear command-line interface, the scanner serves as a fundamental utility for network reconnaissance and initial vulnerability assessments. This project reinforces the understanding of network communication, concurrency, and essential cybersecurity principles related to identifying and securing network entry points.